In [6]:
require "hdf5"
require "optim"
require "nn"

In [7]:
f = hdf5.open("data.hdf5", "r")

X_train = f:read("X_train"):all()
Y_train = f:read("Y_train"):all()
X_valid = f:read("X_valid"):all()
Y_valid = f:read("Y_valid"):all()
X_test = f:read("X_test"):all()
nwords = f:read("nwords"):all()[1]
nclasses = f:read("nclasses"):all()[1]


--sentences
X_valid_sen = f:read("X_valid_sen"):all()
X_test_sen = f:read("X_test_sen"):all()

--MEMM
X_train_MEMM = f:read("X_train_MEMM"):all()
X_valid_MEMM = f:read("X_valid_MEMM"):all()
X_valid_sen_MEMM = f:read("X_valid_sen_MEMM"):all()
nfeaturesMEMM = f:read("nfeaturesMEMM"):all()[1]

C = nclasses

In [8]:
function process(inputs, numwords)
    processed = inputs:clone()
    for i = 1, inputs:size(1) do
        processed[i][2] = inputs[i][2] + numwords
    end
    return processed
end

for i = 1,X_valid_sen_MEMM:size(1) do
    X_valid_sen_MEMM[i] = process(X_valid_sen_MEMM[i], nwords)
end

--X_valid_MEMM = process(X_valid_MEMM, nwords)
X_ = process(X_train_MEMM, nwords)
y = Y_train

In [9]:
addB = nn.Add(nclasses)
lookup = nn.LookupTable(nwords+nclasses,nclasses)
sum = nn.Sum(2)
softmax = nn.LogSoftMax()
h = nn.Sequential()
h:add(lookup)
h:add(sum)
h:add(addB)
mse = nn.CrossEntropyCriterion()

w, dl_dw = h:getParameters()

In [10]:
function feval(w_new)
   
    bsize= batch
    local idx = torch.randperm(X_:size(1)):sub(1,bsize)

    local x = torch.Tensor(bsize, X_:size(2))

    
    --y = y:reshape(Y_train:size(1), 1)
    
    local y_ = torch.Tensor(bsize, 1)

    
    for i=1,bsize do
        x[i] = X_[idx[i]]
        
        y_[i] = y[idx[i]]
    end
    
    y_ = y_:squeeze()
    local inputs = x
    local targets = y_
    -- reset gradients (gradients are always accumulated, to accommodate
    -- batch methods)
    dl_dw:zero()
    -- evaluate the loss function and its derivative with respect to x, given a mini batch
    local prediction = h:forward(inputs)
    local loss_w = mse:forward(prediction, targets)
    h:backward(inputs, mse:backward(prediction, targets))
    
    return loss_w, dl_dw
            
end


In [11]:
batch = 1000

-- cycle on data
for i = 1,30000 do
    -- train a mini_batch of batchSize in parallel
    _, fs = optim.adam(feval,w)

    if i % 1000 == 0 then
        print('loss for iteration ' .. i  .. ' is ' .. fs[1] )
        -- print(sgd_params)
    end
end

loss for iteration 1000 is 1.3506706440324	


loss for iteration 2000 is 0.52696865156655	


loss for iteration 3000 is 0.40270537732585	


loss for iteration 4000 is 0.42298203356749	


loss for iteration 5000 is 0.36056091339807	


...

loss for iteration 26000 is 0.14668870100386	


loss for iteration 27000 is 0.17417253931046	


loss for iteration 28000 is 0.13238834697568	


loss for iteration 29000 is 0.13060489440717	


loss for iteration 30000 is 0.1787002688683	


In [12]:
function gen_score(curr_class, prev_class, input, nword)
    local input1 = input[1]
    local pair = torch.Tensor{{input1, prev_class+nword}}
    local answer = (h:forward(pair))
    answer = nn.LogSoftMax():forward(answer)
    return answer[1][curr_class]
end

function viterbi_gen(sequence, scoring_function, nclass, nword)
    local P_table = torch.ones(nclass, sequence:size(1)):mul(-1e10)
    local BP = torch.zeros(nclass, sequence:size(1))
    local initial = torch.zeros(nclass,1) 
    initial[8] = .99999999
    initial = torch.log(initial / torch.sum(initial))
    P_table[{{},{1,1}}] = initial
    local max_ind = 0
    local max_prob = -1e10
    for i = 2,sequence:size(1) do
        for curr_class_ind = 1,nclass do
            for prev_class_ind = 1,nclass do
                local best_available_P = P_table[curr_class_ind][i]
                local candidate_P = scoring_function(curr_class_ind, prev_class_ind, sequence[i], nword)
                if candidate_P + P_table[prev_class_ind][i-1] > best_available_P then
                    P_table[curr_class_ind][i] = candidate_P + P_table[prev_class_ind][i-1]
                    BP[curr_class_ind][i] = prev_class_ind
                    if P_table[curr_class_ind][i] > max_prob then
                        max_prob = P_table[curr_class_ind][i]
                        max_ind = curr_class_ind
                    end
                end
            end
        end
    end
    
    local classes = torch.zeros(sequence:size(1))
    classes[sequence:size(1)] = max_ind
    for i = sequence:size(1), 2, -1 do
        classes[i-1] = BP[classes[i]][i]
    end
    return classes
end
     
function predict_tags(X)
    local predictions = torch.zeros(X:size(1), X:size(2))

    for i=1,X:size(1) do
        local sen = (X[i]:sub(1,torch.nonzero(X[{{i,i},{},{1,1}}]):size(1)))
        
        
        local p = viterbi_gen(sen, gen_score, C, nwords)
       
        predictions[{{i,i},{1,p:size(1)}}] = p
    end
    return predictions
end

In [13]:

function predict_fscore(p_tags, y_tags)
    local rel_retrived = {[1]=0,[2]=0,[3]=0,[4]=0,[5]=0}--,[6]=0,[7]=0}
    local rel_notretrived = {[1]=0,[2]=0,[3]=0,[4]=0,[5]=0}--,[6]=0,[7]=0}
    local irrel_retrived = {[1]=0,[2]=0,[3]=0,[4]=0,[5]=0}--,[6]=0,[7]=0}
    for i=1,p_tags:size(1) do
        for j=1,p_tags:size(2) do
            for tag=1,5 do        
                if y_tags[i][j] == tag then
                    rel_notretrived[tag] = rel_notretrived[tag] + 1
                    if p_tags[i][j] == tag then
                        rel_retrived[tag] = rel_retrived[tag] + 1
                    else
                        irrel_retrived[tag] = irrel_retrived[tag] + 1
                    end
                end
            end
        end
    end
    local recall = 0
    local precis = 0
    for i=1,5 do
        local r = (rel_retrived[i]/(rel_retrived[i]+rel_notretrived[i]))
        local p = (rel_retrived[i]/(rel_retrived[i]+irrel_retrived[i]))
        --print(i, p, r, 2*p*r/(p+r))
        recall = recall + r
        precis = precis + p
    end
    return recall/5,precis/5
end

function format_valid(valid)
    local y_tags = torch.zeros(X_valid_sen:size(1), X_valid_sen:size(2))
    local sen_idx = 0
    local col_idx = 0
    for i=1,valid:size(1) do
        if valid[i] == 8 then
            sen_idx = sen_idx + 1
            col_idx = 1
        end
        y_tags[sen_idx][col_idx] = valid[i]
        col_idx = col_idx + 1
    end
    return y_tags
end

-- Writes to file in Kaggle prediction format
-- p : prediction tensor
-- fname : output filename

function kaggle_format(p, fname)
    local f = io.open("predictions/" .. fname, "w")
    f:write("ID,Labels\n")
    for i=1,p:size(1) do
        local s = ""
        local t = {[1]="O",[2]="PER",[3]="LOC",[4]="ORG",[5]="MISC",[6]="MISC",[7]="LOC",[8]="O",[9]="O"}
        local prev_tag = 0
        local prev_idx = 0
        for j=2,p:size(2) do
            if p[i][j] == 0 then
                break
            end
            if p[i][j] ~= 1 then
                if prev_tag == 0 then
                    s = s .. t[p[i][j]] .. "-" .. tostring(j-1)
                elseif p[i][j] == prev_tag and prev_idx == j-1 then
                    s = s .. "-" .. tostring(j-1) 
                else
                    s = s .. " " .. t[p[i][j]] .. "-" .. tostring(j-1)
                end
                prev_tag = p[i][j]
                prev_idx = j
            end
        end
        f:write(i .. "," .. s .. "\n")
    end
    f:close()
end

In [687]:
--TAKES A FEW MINUTES TO RUN
preds = predict_tags(X_test_sen)
kaggle_format(preds, "pred_" .. tostring(os.time()) .. "_memm1.csv")

In [ ]:
--TAKES A FEW MINUTES TO RUN
preds = predict_tags(X_valid_sen)
r, p = predict_fscore(preds, y_tags)
print(r, p)
print(2*r*p/(r+p))
